In [25]:
import numpy as np
from PIL import Image
import os


In [26]:
def relu(x):
    return np.maximum(0,x)
def relu_derivative(x):
    return (x>0).astype(float)

In [27]:
def softmax(x):
    exp_x=np.exp(x-np.max(x,axis=1,keepdims=True))
    return exp_x/np.sum(exp_x,axis=1,keepdims=True)


In [28]:
def entropy_loss(y_true,y_pred):
    return -np.mean(np.sum(y_true*np.log(y_pred+1e-8),axis=1))


In [29]:
def forward(x,w1,b1,w2,b2):
    hidden_layer=relu(np.dot(x,w1)+b1)
    output_layer=np.dot(hidden_layer,w2)+b2
    y_pred=softmax(output_layer)
    return hidden_layer,y_pred

In [30]:
def backward(x,y,hidden_layer,y_pred,w2):
    grad_output=(y_pred-y)/y.shape[0]
    grad_w2=np.dot(hidden_layer.T,grad_output)
    grad_b2=np.sum(grad_output,axis=0,keepdims=True)
    grad_hidden=np.dot(grad_output,w2.T)*(hidden_layer>0)
    grad_w1=np.dot(x.T,grad_hidden)
    grad_b1=np.sum(grad_hidden,axis=0,keepdims=True)
    return grad_w1,grad_b1,grad_w2,grad_b2

In [31]:
def train(x,y,inputsize,hiddensize,outputsize,epochs=500,learning_rate=0.001):
    w1=np.random.randn(inputsize,hiddensize)*0.01
    b1=np.zeros((1,hiddensize))
    w2=np.random.randn(hiddensize,outputsize)*0.01
    b2=np.zeros((1,outputsize))
    for epoch in range(epochs):
        hidden_layer,y_pred=forward(x,w1,b1,w2,b2)
        loss=entropy_loss(y,y_pred)
        grad_w1,grad_b1,grad_w2,grad_b2=backward(x,y,hidden_layer,y_pred,w2)
        w1-=learning_rate*grad_w1
        b1-=learning_rate*grad_b1
        w2-=learning_rate*grad_w2
        b2-=learning_rate*grad_b2
        if(epoch+1)%100==0:
            print(f"Epoch{epoch+1}/{epochs},Loss:{loss:4f}")
        return w1,b1,w2,b2

In [32]:
def predict(x,w1,b1,w2,b2):
    hidden_layer=relu(np.dot(x,w1)+b1)
    output_layer=np.dot(hidden_layer,w2)+b2
    probabilities=softmax(output_layer)
    return np.argmax(probabilities,axis=1)

In [41]:
def load_images(folder_path,imagesize=(28,28),return_labels=True):
    images=[]
    labels=[]
    file_paths=[]
    label_map={}
    label_index=0
    for label in os.listdir(folder_path):
        label_path=os.path.join(folder_path,label)
        if os.path.isdir(label_path):
            if label not in label_map:
                label_map[label]=label_index
                label_index+=1
            for file_name in os.listdir(label_path):
                img_path=os.path.join(label_path, file_name)
                img=Image.open(img_path).convert('L').resize(imagesize)
                images.append(np.array(img).flatten())
                labels.append(label_map[label])
                file_paths.append(img_path)
    if return_labels:
        return np.array(images),np.array(labels),file_paths,label_map
    else:
        return np.array(images),file_paths
    



In [42]:
def calculate_accuracy(true_labels,predicted_labels):
    correct=np.sum(true_labels==predicted_labels)
    total=len(true_labels)
    accuracy=(correct/total)*100
    return accuracy

In [43]:
# Paths to datasets
train_path = "Train"
test_path = "Test"
    
# Load training data
X_train, y_train, _, label_map = load_images(train_path)
    
# Load testing data
X_test, test_labels, test_files, _ = load_images(test_path)
    
# Normalize the datasets
X_train = X_train / 255.0
X_test = X_test / 255.0
    
# One-hot encode training labels
num_classes = len(label_map)
y_train_one_hot = np.zeros((y_train.size, num_classes))
y_train_one_hot[np.arange(y_train.size), y_train] = 1
    
# Train the neural network
input_size = X_train.shape[1]
hidden_size = 64
output_size = num_classes
w1, b1, w2, b2 = train(X_train, y_train_one_hot, input_size, hidden_size, output_size)
    
# Predict classes for Tim's Kins
predictions = predict(X_test, w1, b1, w2, b2)
    
# Create a reverse map for class labels
reverse_label_map = {v: k for k, v in label_map.items()}
    
# Print predictions with corresponding file names and class labels
print("\nPredictions for Tim's Kins:")
for i in range(len(predictions)):
    true_label = reverse_label_map[test_labels[i]]
    predicted_label = reverse_label_map[predictions[i]]
    print(f"Image: {os.path.basename(test_files[i])}, True Class: {true_label}, Predicted Class: {predicted_label}")

accuracy = calculate_accuracy(test_labels, predictions)
print(f"Accuracy: {accuracy:.2f}%")


Predictions for Tim's Kins:
Image: -601.png, True Class: Jade, Predicted Class: James
Image: -602.png, True Class: Jade, Predicted Class: James
Image: -603.png, True Class: Jade, Predicted Class: James
Image: -604.png, True Class: Jade, Predicted Class: James
Image: -605.png, True Class: Jade, Predicted Class: James
Image: -606.png, True Class: Jade, Predicted Class: James
Image: -607.png, True Class: Jade, Predicted Class: James
Image: -608.png, True Class: Jade, Predicted Class: James
Image: -609.png, True Class: Jade, Predicted Class: James
Image: -610.png, True Class: Jade, Predicted Class: James
Image: -611.png, True Class: Jade, Predicted Class: James
Image: -612.png, True Class: Jade, Predicted Class: James
Image: -613.png, True Class: Jade, Predicted Class: James
Image: -614.png, True Class: Jade, Predicted Class: James
Image: -615.png, True Class: Jade, Predicted Class: James
Image: -616.png, True Class: Jade, Predicted Class: James
Image: -617.png, True Class: Jade, Predicte